In [4]:
from bs4 import BeautifulSoup
import re
import requests
import json
f = open("API.txt", "r")
key = f.readline()
f.close()

In [18]:
def get_top_200_population_city_cache():
    dump_dict ={}
    url_city = "https://www.michigan-demographics.com/cities_by_population"
    page = requests.get(url_city)
    soup = BeautifulSoup(page.content, 'html.parser')
    city_table = soup.find_all("table", class_="ranklist")
    city_list = [] 
    for i in city_table: 
        name = i.find_all('td')
        for j in name:
            if re.search(r'[~^0-9]',j.text) != None:
                pass
            else:
                text = re.sub('\n','',j.text)
                text = re.sub('          ','',text)
                text = re.sub('      ','',text)
                text = re.sub(' city','',text)
                text = re.sub(' City','',text)
                city_list.append(text)
    top200_city_population = city_list[0:200]
    dump_dict["city_list"] = top200_city_population        
    with open("city.json", "w") as outfile:
        json.dump(dump_dict, outfile)


In [6]:
def get_restaurant(location,offset):
    url='https://api.yelp.com/v3/businesses/search'
    headers = {
    "accept": "application/json",
    "Authorization": key
    }
    params = {'term':'restaurants','location':str(location) + "Michigan","radius" : 20000,"limit":50\
              ,"offset":offset,"locale":"en_US"}

    response = requests.get(url, params=params, headers=headers)
    result = json.loads(response.text)["businesses"]
    return (result)

In [7]:
class location: 
    def __init__(self, name="None",price_tree = []):
        self.name = name
        self.price_tree = price_tree
    def __str__ (self):
        return "location : " + self.name
        

In [8]:
class price: 
    def __init__(self,name="None",rest_tree = []):
        self.name = name
        self.rest_tree = rest_tree
    def __str__ (self):
        return "price : " + self.name

In [9]:
class restaurant:
    def __init__(self, name="None",price= "None",review_count="None",rating = "None",categories = "None" ,id_res= 'None', city="None", location="None",\
                 cord = 'None',url = "None",json = None):
        if json != None:
            self.name = json['name']
            try:
                self.price = json['price']
            except:
                self.price = "None"
            self.review_count = json['review_count']
            self.rating = json['rating']
            self.categories = []
            for tag in json['categories']:
                self.categories.append(tag["title"])
            self.id_res = ['id_res']
            self.city = json['location']['city']
            self.location = json['location']["display_address"]
            self.cord = json['coordinates']
            self.url = json['url']
        else:
            self.name = name
            self.price = price
            self.review_count = review_count
            self.rating = rating
            self.categories = categories
            self.id_res = id_res
            self.city = city
            self.location = location
            self.cord = cord
            self.url = url
    
    def get_categories(self,categories):
        if categories == "None":
            return None
        else:
            str_cate = ""
            for i in categories:
                str_cate = str_cate + i + "; "
        return str_cate
    
    def __str__ (self):
        return "name : " + self.name +"\n"+ "price : " + self.price +"\n"+  "review_count : " + str(self.review_count) +"\n"+ "rating : " + str(self.rating) +"\n"+ "categories : " + self.get_categories(self.categories) +"\n"+ "url : " + self.url +"\n"
                
        

In [10]:
def tree_build_no_cache(city_list):
    tree_root = []
    for city in city_list:
        
        price_1_node = price(name = "$",rest_tree=[])
        price_2_node = price(name = "$$",rest_tree=[])
        price_3_node = price(name = "$$$",rest_tree=[])
        price_4_node = price(name = "$$$$",rest_tree=[])
        price_5_node = price(name = "NA",rest_tree=[])
        
        for offset in range(0,200,50):
            rest_list_json = get_restaurant(city,offset)
            if rest_list_json == []:
                break
            else:
                for rest_item_json in rest_list_json:
                    rest_node = restaurant(json = rest_item_json)
                    if  rest_node.price ==  "$":
                        price_1_node.rest_tree.append(rest_node)
                    elif rest_node.price == "$$":
                        price_2_node.rest_tree.append(rest_node)
                    elif rest_node.price == "$$$":
                        price_3_node.rest_tree.append(rest_node)
                    elif rest_node.price == "$$$$":
                        price_4_node.rest_tree.append(rest_node)
                    elif rest_node.price == "None":
                        price_5_node.rest_tree.append(rest_node)
        
        city_node = location(name = city)
        city_node.price_tree = [price_1_node,price_2_node,price_3_node,price_4_node,price_5_node]
        tree_root.append(city_node)
    return tree_root

In [11]:
def cache_city_data(city_list):
    dump_dict ={}
    for city in city_list:
        temp_list = []
        for offset in range(0,300,50):
            rest_list_json = get_restaurant(city,offset)
            if rest_list_json == []:
                break
            else:
                temp_list = temp_list + rest_list_json
        dump_dict[str(city)] = temp_list
            
    with open("restaurant.json", "w") as outfile:
        json.dump(dump_dict, outfile)
            

In [31]:
def load_city(city_file):
    f = open(city_file)
    cache_file = json.load(f)
    f.close()
    return(cache_file["city_list"])

In [12]:
def tree_build_with_cache(city_list,cache_file):
    f = open(cache_file)
    cache_file = json.load(f)
    f.close()
    
    tree_root = []
    for city in city_list:
        
        price_1_node = price(name = "$",rest_tree=[])
        price_2_node = price(name = "$$",rest_tree=[])
        price_3_node = price(name = "$$$",rest_tree=[])
        price_4_node = price(name = "$$$$",rest_tree=[])
        price_5_node = price(name = "NA",rest_tree=[])
        
        cache_file
        rest_list_json = cache_file[str(city)]
        
        for rest_item_json in rest_list_json:
            rest_node = restaurant(json = rest_item_json)
            if  rest_node.price ==  "$":
                price_1_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$":
                price_2_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$$":
                price_3_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$$$":
                price_4_node.rest_tree.append(rest_node)
            elif rest_node.price == "None":
                price_5_node.rest_tree.append(rest_node)
        
        city_node = location(name = city)
        city_node.price_tree = [price_1_node,price_2_node,price_3_node,price_4_node,price_5_node]
        tree_root.append(city_node)
    return tree_root

In [15]:
def question(city_list):
    
    print("Hello! Welcome to the restaurant recommendation app! I will try to give you some great dining choices!")
    print("First please answer some questions.")
    print()
    print('Here are some more populated city.')
    print("Detroit; Grand Rapids; Sterling Heights;Ann Arbor; Lansing")
    
    while True:
        print("Please tell me the city you are at.")
        print("Please captitallize the first letter of each word of the city.")
        city_answer = input('Your Location: ')
        print()
        if city_answer not in city_list:
            print('City not found, maybe you can try the township of the city.')
            print("Please captitallize the first letter of the township name and add township in the end.")
        else:
            break
            
    while True:
        print('What is the your price range for the meal?')
        print("You can enter one to four dollar sign.")
        price_answer = input('Your Budget: ')
        
        if price_answer != "$" and price_answer != "$$" and price_answer != "$$$" and price_answer != "$$$$":
            print("\n")
            print("Please try again")
        else:
            break
    print('\n')        
    print('What cuisine or kind of food you are interested in')
    print('Here is some example: American; Asian; Italian; Mexican; Chinese; Middle Eastern; Korean')
    print('Here is More example: Breakfast; Pizza; Cafes; Sushi')
    print('Please try to use a single word')
    interest_answer = input('Your interest: ').title()

  
    while True:
        print("\n")
        print('What is the minimal rating score you can accept')
        print("You can input from 1 to 5")
        score_answer = input('Your score: ')
        try:
            score_answer = float(score_answer)
            if score_answer < 1 or score_answer > 5:
                print('Please input a valid score')
            else:
                break
        except:
            print('Please input a valid score')
            
       
    
    print('If you want to know the distance from the restaurants to you location, please enter your address.')
    print('else you can enter N to skip the part')
    location_answer = input("Your location: ")
    
    return [city_answer,price_answer,interest_answer,score_answer,location_answer]


In [ ]:
a = get_top_200_population_city()
b = tree_build_with_cache(a,"restaurant.json")

In [ ]:
def search(question,tree_root):
    
    

In [16]:
c = interface(a)

Hello! Welcome to the restaurant recommendation app! I will try to give you some great dining choices!
First please answer some questions.

Here are some more populated city.
Detroit; Grand Rapids; Sterling Heights;Ann Arbor; Lansing
Please tell me the city you are at.
Please captitallize the first letter of each word of the city.
Your Location: Ann Arbor

What is the your price range for the meal?
You can enter one to four dollar sign.
Your Budget: $$


What cuisine or kind of food you are interested in
Here is some example: American; Asian; Italian; Mexican; Chinese; Middle Eastern; Korean
Here is More example: Breakfast; Pizza; Cafes; Sushi
Please try to use a single word
Your interest: Sushi


What is the minimal rating score you can accept
You can input from 1 to 5
Your score: 3
If you want to know the distance from the restaurants to you location, please enter your address.
else you can enter N to skip the part
Your location: N


In [17]:
c

['Ann Arbor', '$$', 'Sushi', 3.0, 'N']